In [78]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig

from dotenv import load_dotenv

load_dotenv()

class State(TypedDict):
    messages: Annotated[list, add_messages]
    
tavily_search_tool = TavilySearchResults(
max_results=1,
include_answer=True,
include_raw_content=True,
include_domains=["github.io", "wikidocs.net"],
)

tools = [tavily_search_tool]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)


def call_llm(state: State):
    answer = llm_with_tools.invoke(state["messages"])
    return {"messages": [answer]}


tools = ToolNode(tools=[tavily_search_tool])

graph_builder = StateGraph(State)

graph_builder.add_node("call_llm", call_llm)
graph_builder.add_node("tools", tools)

graph_builder.add_conditional_edges(
    "call_llm",
    tools_condition,
)

graph_builder.add_edge("tools", "call_llm")
graph_builder.add_edge(START, "call_llm")
graph_builder.add_edge("call_llm", END)

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "1"})

In [68]:
input = {"messages": [("human", "안녕하세요. 스타듀밸리 게임에 대해서 검색해주세요.")]}
events = graph.stream(input=input, config=config)

for event in events:
    for key, value in event.items():
        print(f"key = {key}, value = {value}")

key = call_llm, value = {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qiaIiktwUzktfuDHF7m3QORB', 'function': {'arguments': '{"query":"스타듀밸리 게임"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 554, 'total_tokens': 579, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-13b50412-8223-4a10-9ffc-0111baed2759-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스타듀밸리 게임'}, 'id': 'call_qiaIiktwUzktfuDHF7m3QORB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 554, 'output_tokens': 25, 'total_tokens': 579, 'input_token_details': {'au

`event.items()` 메서드로 각 단계 별로 key와 value를 받아올 수 있는데,

key는 노드의 이름이 `str`로 저장되어 있고 value에는 해당 노드 단계의 출력 값이 Dict로 저장되어 있다.
(기본 값인 stream_mode="updates" 기준)


In [ ]:
input = {"messages": [("human", "안녕하세요. 스타듀밸리 게임에 대해서 검색해주세요.")]}
events = graph.stream(input=input, config=config)

for event in events:
    for key, value in event.items():
        if "messages" in value:
            print(value["messages"][-1].pretty_print())

================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_09TazPRc1fRp06BxqRvNjhA6)
 Call ID: call_09TazPRc1fRp06BxqRvNjhA6
  Args:
    query: 스타듀밸리 게임
None
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://mouseypounds.github.io/stardew-checkup/", "content": "19 Mar 2024 - v5.0 - Support for Stardew Valley 1.6 22 Dec 2020 - v4.0 - Output Preferences and initial support for Stardew Valley 1.5 1 Dec 2019 - v3.0.2 - Fixed bug in Joja development summary 26 Nov 2019 - v3.0 - Support for Stardew Valley 1.4 22 June 2019 - v2.5 - Added Introductions quest summary to Social 30 Jan 2019 - v2.4 - Improved support for iOS save files 9 Mar 2018 - v1.7 - Added \"heart events\" to Social friendship summary 29 Jan 2018 - v1.5 - Added full friendship point summary to Social section 28 June 2017 - v1.3 - Added navigation links along right side (only v

`pretty_print()` 메서드로 메시지를 예쁘게 출력해 볼 수 있다.


In [70]:
print(graph.channels.keys())

dict_keys(['messages', '__start__', 'call_llm', 'tools', 'branch:__start__:__self__:call_llm', 'branch:__start__:__self__:tools', 'branch:call_llm:__self__:call_llm', 'branch:call_llm:__self__:tools', 'branch:tools:__self__:call_llm', 'branch:tools:__self__:tools', 'start:call_llm', 'branch:call_llm:tools_condition:tools'])


`channels.keys()` 메서드로 출력되는 항목의 Key를 확인 할 수 있다.

In [79]:
input = {"messages": [("human", "안녕하세요. 스타듀밸리 게임에 대해서 검색해주세요.")]}
events = graph.stream(input=input, config=config, output_keys=["messages"])

for event in events:
    for key, value in event.items():
        if "messages" in value:
            print(value["messages"][-1].content)


[{"url": "https://mouseypounds.github.io/stardew-checkup/", "content": "19 Mar 2024 - v5.0 - Support for Stardew Valley 1.6 22 Dec 2020 - v4.0 - Output Preferences and initial support for Stardew Valley 1.5 1 Dec 2019 - v3.0.2 - Fixed bug in Joja development summary 26 Nov 2019 - v3.0 - Support for Stardew Valley 1.4 22 June 2019 - v2.5 - Added Introductions quest summary to Social 30 Jan 2019 - v2.4 - Improved support for iOS save files 9 Mar 2018 - v1.7 - Added \"heart events\" to Social friendship summary 29 Jan 2018 - v1.5 - Added full friendship point summary to Social section 28 June 2017 - v1.3 - Added navigation links along right side (only visible after a save is loaded)"}]
스타듀밸리(Stardew Valley)는 농장 경영과 삶의 시뮬레이션을 결합한 인기 게임입니다. 플레이어는 작은 농장을 물려받고, 이를 발전시켜 나가면서 다양한 활동을 수행할 수 있습니다. 농사, 어업, 광산 탐험, 동물 기르기, 마을 주민들과의 상호작용 등이 핵심 요소입니다.

더 자세한 정보를 원하시면 [여기를 클릭해주세요](https://mouseypounds.github.io/stardew-checkup/). 이 링크는 스타듀밸리의 업데이트와 관련된 정보를 제공합니다.
